# Model set-up using the transition object

The most important part of setting up the model is to correctly define the set ODEs, which is based solely on the classes defined in `transition`. These transitions describe how members of one state can move into another state. We choose to define the models using transitions because it enables the computer to do our book-keeping, therefore reducing the error of, for example, including a flow out of one state, but forgetting to include it in the recipient state. (This does not apply to birth and death processes.) 


All transitions that get fed into the ODE system need to be defined as a transition object, `Transition`. It takes a total of four input arguments:

1.  The origin state
2.  Equation that describe the process
3.  The type of transition
4.  The destination state

where the first three are mandatory. To demonstrate, we go back to the SIR model defined previously in the section {doc}`sir`. Recall that the set of ODEs are

$$\begin{aligned}
 \frac{d S}{d t} &= - beta SI \\
\frac{d I}{d t} &= beta SI - \gamma I \\
\frac{d R}{d t} &= \gamma I.
\end{aligned}$$

We can define the set of ODEs, as seen previously, via

In [ ]:
from pygom import Transition, TransitionType, common_models

ode1 = Transition(origin='S', equation='-beta*S*I', transition_type=TransitionType.ODE)

ode2 = Transition(origin='I', equation='beta*S*I - gamma*I', transition_type=TransitionType.ODE)

ode3 = Transition(origin='R', equation='gamma*I', transition_type=TransitionType.ODE)


Note that we need to state explicitly the type of equation we are
inputting, which is simply of type **ODE** in this case. We can confirm
this has been entered correctly by putting it into `DeterministicOde`,

In [ ]:
from pygom import DeterministicOde

stateList = ['S', 'I', 'R']

paramList = ['beta', 'gamma']

model = DeterministicOde(stateList, paramList, ode=[ode1, ode2, ode3])


and then checking it.


In [ ]:
model.get_ode_eqn()

Now we are going to show the different ways of defining the same set of
ODEs.


(transition:defining-the-equations)=
## Defining the equations

We first recognize that the set of ODEs defining the SIR model are the result of
two transitions,

$$\begin{aligned}
S \rightarrow I &= \beta SI \\
I \rightarrow R &= \gamma I
\end{aligned}$$

where $S \rightarrow I$ denotes a transition from state $S$ to state
$I$. Therefore, we can define our model by these two transitions,
but they need to be passed as the `transition`
argument instead of the `ode` argument of `DeterministicOde` or `SimulateOde`.

```{note}
We are initializing the model using the `SimulateOde` class, rather than `DeterministicOde`, because the stochastic implementation has more available operations on transitions.
```

In [ ]:
from pygom import SimulateOde

t1 = Transition(origin='S', destination='I', equation='beta*S*I', transition_type=TransitionType.T)

t2 = Transition(origin='I', destination='R', equation='gamma*I', transition_type=TransitionType.T)

modelTrans = SimulateOde(stateList, paramList, transition=[t1, t2])

modelTrans.get_ode_eqn()


We can see that the resulting ODE is exactly the same, as expected. The
transition matrix that defines this process can be visualized
using graphviz. Because only certain renderers permit the use of sub and
superscript, operators such as $**$ are left as they are in the
equation.

In [ ]:
modelTrans.get_transition_matrix()

In [ ]:
import matplotlib.pyplot as plt
# TODO why are two images produced? issue #75
modelTrans.get_transition_graph(show=True)

```{warning}
The execution will error if the incorrect `TransitionType` is used against the wrong argument.

`modelTrans = DeterministicOde(stateList, paramList, ode=[t1, t2])`

Here the error occurs because `t1` and `t2` used `transition_type=TransitionType.T` argument, but `DeterministicOde` is expecting a `TransitionType.ODE` argument.

Similarly `DeterministicOde(stateList, paramList, transition=[ode1, ode2, ode3])` would fail.

This therefore forces us to construct our model carefully.
```

The third option is to reframe the system as a set of birth processes, using `transition_type=TransitionType.B`. For this simple example, this formulation takes a similar form to defining using ODE equations.


In [ ]:
birth1 = Transition(origin='S', equation='-beta*S*I', transition_type=TransitionType.B)

birth2 = Transition(origin='I', equation='beta*S*I - gamma*I', transition_type=TransitionType.B)

birth3 = Transition(origin='R', equation='gamma*I', transition_type=TransitionType.B)

modelBirth = DeterministicOde(stateList, paramList, birth_death=[birth1, birth2, birth3])

modelBirth.get_ode_eqn()


Alternatively, we can use the negative of the equation to configure the ODEs to represent death processes. Since the death process is the removal of a flow, we take the negative of the birth process alongside using `transition_type=TransitionType.D`.

In [ ]:
death1 = Transition(origin='S', equation='beta*S*I', transition_type=TransitionType.D)

birth2 = Transition(origin='I', equation='beta*S*I - gamma*I', transition_type=TransitionType.B)

death3 = Transition(origin='R', equation='-gamma*I', transition_type=TransitionType.D)

modelBD = DeterministicOde(stateList, paramList, birth_death=[death1, birth2, death3])

modelBD.get_ode_eqn()


We can see that all four approaches have yielded the same set of ODEs at the end.


## Model Addition

Because we allow the separation of transitions between states and birth/death processes, the birth/death processes can be added later on. The following example takes the model that was defined using transitions (`modelTrans`) and includes a birth process to the $S$ state, and death processes to the $S$ and $I$ states.

In [ ]:
modelBD2 = modelTrans

modelBD2.param_list = paramList + ['mu', 'B']

birthDeathList = [Transition(origin='S', equation='B', transition_type=TransitionType.B),  
                  Transition(origin='S', equation='mu*S', transition_type=TransitionType.D), 
                  Transition(origin='I', equation='mu*I', transition_type=TransitionType.D)
                  ]

modelBD2.birth_death_list = birthDeathList

modelBD2.get_ode_eqn()


This demonstrates that we can approach our in stages. Start off with a standard closed system using `TransitionType.T`, and then extend it with additional flows that interact with the populations' surrounding environments using `TransitionType.B` or `TransitionType.D`.

## Transition type summary

In summary, there are four different types of transitions allowed. These are B, D, ODE and T, which are defined in an enum class also located in `transition`.

In [ ]:
from pygom import transition

for i in transition.TransitionType:  
    print(str(i) + " = " + i.value)


Each birth process is added to the origin state, while each death
process is deducted from the origin state (alternatively added to the state after
multiplying the flow with a negative sign). An ODE type is also added to the state, but we forbid the number of input ODEs to be greater than the number of states inputted. These strict definitions should help us to improve the bookeeping of states and flows when we have models of greater complexity.